In [1]:
import jax
import jax.numpy as jnp
import haiku as hk
import optax
import sys
from time import sleep
import json
import pexpect
import re
import timeit


In [2]:
HOLPATH = "/home/sean/Documents/PhD/HOL4/HOL/bin/hol --maxheap=256"
TARGET_THEORIES = ["pred_set"] #["arithmetic"]#["list"] #["rich_list"] # ["integer"] #["arithmetic"] # ["rich_list"] #["pred_set"]

In [3]:
with open("typed_database.json") as f:
    database = json.load(f)

In [4]:
#goals = [value[4] for key, value in database.items() if value[3] == "thm" and value[0] in TARGET_THEORIES]

In [5]:
#goals

In [6]:
def get_polish(raw_goal):
        goal = construct_goal(raw_goal)
        process.sendline(goal.encode("utf-8"))
        process.expect("\r\n>")
        process.sendline("val _ = set_term_printer (HOLPP.add_string o pt);".encode("utf-8"))
        process.expect("\r\n>")
        process.sendline("top_goals();".encode("utf-8"))
        process.expect("val it =")
        process.expect([": goal list", ":\r\n +goal list"])

        polished_raw = process.before.decode("utf-8")
        polished_subgoals = re.sub("“|”","\"", polished_raw)
        polished_subgoals = re.sub("\r\n +"," ", polished_subgoals)

        # print("content:{}".format(subgoals))
        # exit()
        pd = eval(polished_subgoals)
        
        process.expect("\r\n>")
        process.sendline("drop();".encode("utf-8"))
        process.expect("\r\n>")
        process.sendline("val _ = set_term_printer default_pt;".encode("utf-8"))
        process.expect("\r\n>")

        data = [{"polished":{"assumptions": e[0][0], "goal":e[0][1]},
                 "plain":{"assumptions": e[1][0], "goal":e[1][1]}}
                for e in zip(pd, [([], raw_goal)])]
        return data # list(zip(pd, [([], raw_goal)]))f

def construct_goal(goal):
    s = "g " + "`" + goal + "`;"
    return s

In [7]:
# process = pexpect.spawn(HOLPATH)

# theories = ["listTheory", "bossLib"]

# print("Importing theories...")
# process.sendline("val _ = HOL_Interactive.toggle_quietdec();".encode("utf-8"))
# for i in theories:
#     process.sendline("open {};".format(i).encode("utf-8"))

In [8]:

# # remove built-in simp lemmas
# print("Removing simp lemmas...")
# process.sendline("delsimps [\"HD\", \"EL_restricted\", \"EL_simp_restricted\"];")
# #process.sendline("delsimps {};".format(dels))
# #process.sendline("delsimps {};".format(dels2))
# # process.sendline("delsimps {};".format(dels3))
# sleep(4)
# # load utils
# print("Loading modules...")
# process.sendline("use \"helper.sml\";")
# sleep(5)
# # process.sendline("val _ = load \"Timeout\";")
# print("Configuration done.")
# process.expect('\r\n>')
# # process.readline()
# process.sendline("val _ = HOL_Interactive.toggle_quietdec();".encode("utf-8"))

# # consumes hol4 head
# process.expect('\r\n>')
# #print (process.read())
# # setup the goal
# goal = goals[0]
                      
#  # a pair of polished goal list and original goal list
# fringe = get_polish(goal)

# #scripts = []
# goal = construct_goal(goal)
# process.sendline(goal.encode("utf-8"))        
# print("Initialization done. Main goal is:\n{}.".format(goal))


In [9]:
# print (fringe)

In [10]:
tactic_zero_path = "/home/sean/Documents/PhD/git/repo/PhD/tacticzero/holgym/"


In [11]:
test_history = [{'content': [{'polished': {'assumptions': [], 'goal': '@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Vl Clist$NIL @ @ Cmin$= Vl Clist$NIL'}, 'plain': {'assumptions': [], 'goal': 'REVERSE l = [] ⇔ l = []'}}], 'parent': None, 'goal': None, 'by_tactic': '', 'reward': None}, {'content': [{'polished': {'assumptions': [], 'goal': '@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Clist$NIL Clist$NIL @ @ Cmin$= Clist$NIL Clist$NIL'}, 'plain': {'assumptions': [], 'goal': 'REVERSE [] = [] ⇔ [] = []'}}, {'polished': {'assumptions': ['@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Vl Clist$NIL @ @ Cmin$= Vl Clist$NIL'], 'goal': '@ Cbool$! | Vh @ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE @ @ Clist$CONS Vh Vl Clist$NIL @ @ Cmin$= @ @ Clist$CONS Vh Vl Clist$NIL'}, 'plain': {'assumptions': ['REVERSE l = [] ⇔ l = []'], 'goal': '∀h. REVERSE (h::l) = [] ⇔ h::l = []'}}], 'parent': 0, 'goal': 0, 'by_tactic': 'Induct_on `l`', 'reward': 0.1}, {'content': [{'polished': {'assumptions': ['@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Vl Clist$NIL @ @ Cmin$= Vl Clist$NIL'], 'goal': '@ Cbool$! | Vh @ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE @ @ Clist$CONS Vh Vl Clist$NIL @ @ Cmin$= @ @ Clist$CONS Vh Vl Clist$NIL'}, 'plain': {'assumptions': ['REVERSE l = [] ⇔ l = []'], 'goal': '∀h. REVERSE (h::l) = [] ⇔ h::l = []'}}], 'parent': 1, 'goal': 0, 'by_tactic': 'fs[EL_simp_restricted, LENGTH_ZIP, SUM_eq_0, LENGTH_EQ_NUM, FILTER_NEQ_NIL]', 'reward': 0.2}, {'content': [], 'parent': 2, 'goal': 0, 'by_tactic': 'fs[APPEND_LENGTH_EQ, LIST_REL_rules, FILTER_NEQ_NIL, REVERSE_11, LENGTH_REVERSE]', 'reward': 5}]



In [12]:
def gather_encoded_content_(history, encoder):
    # figure out why this is slower than tests
    # figured out: remember to do strip().split()
    fringe_sizes = []
    contexts = []
    reverted = []
    for i in history:
        c = i["content"]
        contexts.extend(c)
        fringe_sizes.append(len(c))
    for e in contexts:
        g = revert_with_polish(e)
        reverted.append(g.strip().split())
    out = []
    sizes = []
    for goal in reverted:
        out_, sizes_ = encoder.encode([goal])
        out.append(torch.cat(out_.split(1), dim=2).squeeze(0))
        sizes.append(sizes_)
        
    # s1 = timeit.default_timer()
    #out, sizes = batch_encoder.encode(reverted)
    
    representations = out
    # merge two hidden variables
    #representations = torch.cat(out.split(1), dim=2).squeeze(0)
    # print(representations.shape)
    # s2 = timeit.default_timer()    
    # print(s2-s1)

    return representations, contexts, fringe_sizes

In [13]:
# ARG_LEN = 5 
# EXCLUDED_THEORIES = ["min"] #["min", "bool"]
# def replay_known_proof(known_history, arg_len=ARG_LEN):
#     print("\nReplaying a known proof of {} ...".format(known_history[0]["content"][0]["plain"]["goal"]))

#     # compute the encoded fact pool at the beginning of each episode
#     polished_goal = known_history[0]["content"][0]["polished"]["goal"]
#     allowed_theories = parse_theory(polished_goal)
#     allowed_theories = [t for t in allowed_theories if t not in EXCLUDED_THEORIES]
    
#     try:
#         allowed_arguments_ids = []
#         candidate_args = []
#         goal_theory = database[polished_goal][0]
#         for i,t in enumerate(database):
#             if database[t][0] in allowed_theories and (database[t][0] != goal_theory or int(database[t][2]) < int(database[polished_goal][2])):
#                 allowed_arguments_ids.append(i)
#                 candidate_args.append(t)

#     except:
#         allowed_arguments_ids = []
#         candidate_args = []
#         for i,t in enumerate(database):
#             if database[t][0] in allowed_theories:
#                 allowed_arguments_ids.append(i)
#                 candidate_args.append(t)
#         print("Theorem not found in database.")

#     #true_resulting_fringe = known_history[t+1]

#     #print (known_history)

#     representations, context_set, fringe_sizes = gather_encoded_content_(known_history, batch_encoder)

#     return representations, context_set, fringe_sizes

    
def parse_theory(pg):
    theories = re.findall(r'C\$(\w+)\$ ', pg)
    theories = set(theories)
    for th in EXCLUDED_THEORIES:
        theories.discard(th)
    return list(theories)

def revert_with_polish(context):
    target = context["polished"]
    assumptions = target["assumptions"]
    goal = target["goal"]
    for i in reversed(assumptions): 
        goal = "@ @ Dmin$==> {} {}".format(i, goal)
    return goal 

def split_by_fringe(goal_set, goal_scores, fringe_sizes):
    # group the scores by fringe
    fs = []
    gs = []
    counter = 0
    for i in fringe_sizes:
        end = counter + i
        fs.append(goal_scores[counter:end])
        gs.append(goal_set[counter:end])
        counter = end
    return gs, fs

In [14]:
from new_env import *


with open("include_probability.json") as f:
    #database  of all theories up to probability?
    #keys are polish notation expressions, values give list with [theory/library, theory_name, number in lib, def/thm, utf encoded expression]
    database = json.load(f)

#all theories in database
#TARGET_THEORIES = ["probability", "martingale", "lebesgue", "borel", "real_borel", "sigma_algebra","util_prob", "fcp", "indexedLists", "rich_list", "list", "pred_set","numpair", "basicSize", "numeral", "arithmetic", "prim_rec", "num","marker", "bool", "min", "normalForms", "relation", "sum", "pair", "sat","while", "bit", "logroot", "transc", "powser", "lim", "seq", "nets","metric", "real", "realax", "hreal", "hrat", "quotient_sum", "quotient","res_quan", "product", "iterate", "cardinal", "wellorder","set_relation", "derivative", "real_topology"]
TARGET_THEORIES = ["bool", "min", "list"]
GOALS = [(key, value[4]) for key, value in database.items() if value[3] == "thm" and value[0] in TARGET_THEORIES]

len(GOALS)

with open("polished_def_dict.json") as f:
    defs = json.load(f)

fact_pool = list(defs.keys())



#parse theory
g = GOALS[19][1]


print (g)

env = HolEnv(g)


theories = re.findall(r'C\$(\w+)\$ ', GOALS[19][0])
theories = set(theories)
theories = list(theories)

print (theories)

allowed_theories = theories

goal_theory = g

print ("Target goal: {}".format(g))
try:
    allowed_arguments_ids = []
    candidate_args = []
    goal_theory = g#database[polished_goal][0] # plain_database[goal][0]
    for i,t in enumerate(database):
        if database[t][0] in allowed_theories and (database[t][0] != goal_theory or int(database[t][2]) < int(database[polished_goal][2])):
            allowed_arguments_ids.append(i)
            candidate_args.append(t)

    env.toggle_simpset("diminish", goal_theory)
    print("Removed simpset of {}".format(goal_theory))

except:
    allowed_arguments_ids = []
    candidate_args = []
    for i,t in enumerate(database):
        if database[t][0] in allowed_theories:
            allowed_arguments_ids.append(i)
            candidate_args.append(t)
    print("Theorem not found in database.")

print ("Number of candidate facts to use: {}".format(len(candidate_args)))


# print("Facts: {}".format(len(allowed_arguments_ids)))

# state = env.history

#encoded database for representations of all terms in database (easier than computing batch encoding for arguments each iteration)

encoded_database = torch.load('encoded_include_probability.pt')

encoded_fact_pool = torch.index_select(encoded_database, 0, torch.tensor(allowed_arguments_ids))

print (env.history)

/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


∀(h :α) (t :α list). TL (h::t) = t
Importing theories...
Loading modules...
Configuration done.
['list', 'bool', 'min']
Target goal: ∀(h :α) (t :α list). TL (h::t) = t
Removing simp lemmas from ∀(h :α) (t :α list). TL (h::t) = t
Removed simpset of ∀(h :α) (t :α list). TL (h::t) = t
Number of candidate facts to use: 686
[{'content': [{'polished': {'assumptions': [], 'goal': '@ C$bool$ ! | Vh @ C$bool$ ! | Vt @ @ C$min$ = @ C$list$ TL @ @ C$list$ CONS Vh Vt Vt'}, 'plain': {'assumptions': [], 'goal': '∀(h :α) (t :α list). TL (h::t) = t'}}], 'parent': None, 'goal': None, 'by_tactic': '', 'reward': None}]


In [15]:
# import os
# import argparse
# import logging
# import timeit
# import torch

# import seq2seq
# #from seq2seq.evaluator import BatchPredictor
# #from seq2sebq.util.checkpoint import Checkpoint

# from batch_predictor import BatchPredictor
# from checkpoint import Checkpoint

# #checkpoint_path = "models/2021_02_21_15_46_04" # 98% accuracy model, up to probability theory
# checkpoint_path = "models/2020_04_26_20_11_28" # 95% accuracy model, core theories + integer + sorting

# #checkpoint_path = "models/2021_02_22_16_07_03" # 97-98% accuracy model, up to and include probability theory#
# #checkpoint_path = "models/2020_09_24_23_38_06" # 98% accuracy model, core theories + integer + sorting | separate theory tokens


# # logging.info("loading checkpoint from {}".format(checkpoint_path))

# checkpoint = Checkpoint.load(checkpoint_path)
# seq2seq = checkpoint.model
# input_vocab = checkpoint.input_vocab
# output_vocab = checkpoint.output_vocab

# batch_encoder = BatchPredictor(seq2seq, input_vocab, output_vocab)
# batch_encoder

In [16]:
import os
import argparse
import logging
import timeit
import torch

import seq2seq
#from seq2seq.evaluator import BatchPredictor
#from seq2sebq.util.checkpoint import Checkpoint

from batch_predictor import BatchPredictor
from checkpoint import Checkpoint



    
    #checkpoint_path = "models/2020_04_22_20_36_50" # 91% accuracy model, only core theories
#checkpoint_path = "models/2020_04_26_20_11_28" # 95% accuracy model, core theories + integer + sorting
#checkpoint_path = "models/2020_09_24_23_38_06" # 98% accuracy model, core theories + integer + sorting | separate theory tokens
#checkpoint_path = "models/2020_11_28_16_45_10" # 96-98% accuracy model, core theories + integer + sorting + real | separate theory tokens

checkpoint_path = "models/2020_12_04_03_47_22" # 97% accuracy model, core theories + integer + sorting + real + bag | separate theory tokens

#checkpoint_path = "models/2021_02_21_15_46_04" # 98% accuracy model, up to probability theory

#checkpoint_path = "models/2021_02_22_16_07_03" # 97-98% accuracy model, up to and include probability theory

checkpoint = Checkpoint.load(checkpoint_path)
print (checkpoint)
seq2seq = checkpoint.model
input_vocab = checkpoint.input_vocab
output_vocab = checkpoint.output_vocab

batch_encoder_ = BatchPredictor(seq2seq, input_vocab, output_vocab)



jax_reps, context_set, fringe_sizes = gather_encoded_content_(env.history, batch_encoder_)



print (jax_reps, context_set, fringe_sizes)

[tensor([[-0.3994, -0.0156,  0.6180,  0.9910,  0.9908,  0.4474, -0.1244,  0.7957,
         -0.9979,  0.9996, -0.8560,  0.5563, -0.9832, -0.0475, -0.6017,  0.9556,
          0.9066,  0.2458, -0.7248,  0.3159, -0.4685,  0.0438,  0.7034,  0.9882,
          0.9988, -0.3311, -0.7088,  0.5712,  0.5283,  0.9990,  0.4139, -0.1177,
          0.9468,  0.4267,  0.4278,  0.9838,  0.0524,  0.8602,  0.5130, -0.3157,
          0.4570, -0.9315, -0.6033, -0.7443,  0.5236, -0.9620, -0.6450, -0.9345,
         -0.3024,  0.9209,  0.2879,  0.6390,  0.8362, -0.2326,  0.9998,  0.9946,
          0.9920, -0.3666,  0.5162,  0.9952, -0.0544, -0.9910,  0.6673, -0.1317,
         -0.5496, -0.0348, -0.3363, -0.1494,  0.1450, -0.8604,  0.4788, -0.9989,
          0.0788,  0.0276, -0.5614,  0.4859,  0.8501, -0.9862, -0.7372,  0.7747,
         -0.0035,  0.6698, -0.9900,  0.9416, -0.3604, -0.3198, -0.8628,  0.9515,
          0.4072, -0.4891,  0.3294, -0.1489, -0.8057,  0.6508, -0.3875, -0.1206,
         -0.4258, -0.5316, 

/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'seq2seq.models.EncoderRNN.EncoderRNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the origina

In [17]:
#jax_reps, context_set, fringe_sizes = replay_known_proof(test_history)

In [18]:
print (context_set, fringe_sizes)

[{'polished': {'assumptions': [], 'goal': '@ C$bool$ ! | Vh @ C$bool$ ! | Vt @ @ C$min$ = @ C$list$ TL @ @ C$list$ CONS Vh Vt Vt'}, 'plain': {'assumptions': [], 'goal': '∀(h :α) (t :α list). TL (h::t) = t'}}] [1]


In [19]:
from policy_networks import *
import policy_networks

In [20]:
init_context, apply_context = hk.transform(policy_networks._context_forward)
rng_key = random.PRNGKey(100)

#convert to jax
jax_reps = jnp.stack([jnp.array(jax_reps[i][0]) for i in range(len(jax_reps))])


context_params = init_context(rng_key, jax_reps)
#apply_context = jax.jit(apply_context)

jax_reps

DeviceArray([[-0.3993603 , -0.01555056,  0.61800957,  0.99099714,
               0.9907693 ,  0.4473821 , -0.12441963,  0.7957076 ,
              -0.9978648 ,  0.9995919 , -0.85603034,  0.5563285 ,
              -0.98323023, -0.04748891, -0.601659  ,  0.9556234 ,
               0.90661794,  0.24579544, -0.72479576,  0.31592473,
              -0.4684776 ,  0.04382285,  0.7034173 ,  0.98821664,
               0.99876744, -0.33105958, -0.7087546 ,  0.57119954,
               0.5282592 ,  0.99903846,  0.41393578, -0.1176954 ,
               0.9468151 ,  0.42671782,  0.42777938,  0.9838208 ,
               0.0524292 ,  0.86019003,  0.5129815 , -0.31572363,
               0.45703578, -0.931481  , -0.6032594 , -0.74428594,
               0.5235599 , -0.9620007 , -0.64496315, -0.934458  ,
              -0.3024236 ,  0.9208751 ,  0.2878877 ,  0.6390267 ,
               0.8362171 , -0.23259316,  0.99980056,  0.99455684,
               0.9919802 , -0.3665627 ,  0.5162257 ,  0.9952254 ,
          

In [21]:
# context_scores = apply_context(context_params, rng_key, jax_reps)
# print (context_scores)

In [22]:

#function to give the log probability of pi(f | s) so gradient can be computed directly
#also returns sampled index and contexts to determine goal to give tactic network
def sample_fringe(context_params, context_net, rng_key, jax_reps, context_set, fringe_sizes):
    context_scores = context_net(context_params, rng_key, jax_reps)
    contexts_by_fringe, scores_by_fringe = split_by_fringe(context_set, context_scores, fringe_sizes)
    fringe_scores = []
    print (fringe_scores)
    for s in scores_by_fringe:
        fringe_score = jnp.sum(s)
        fringe_scores.append(fringe_score)
    #TODO some fringes can be empty, but still give value 0 which assigns nonzero probability?
    fringe_scores = jnp.stack(fringe_scores)
    fringe_probs = jax.nn.softmax(fringe_scores)

    #samples, gives an index (looks like it does gumbel softmax under the hood to keep differentiability?)
    sampled_idx = random.categorical(rng_key,fringe_probs)

    prob = fringe_probs[sampled_idx]
    log_prob = jnp.log(prob)
    return log_prob, (sampled_idx, contexts_by_fringe)
                                                           
grad_log_context, (fringe_idx, contexts_by_fringe) = jax.grad(sample_fringe, has_aux=True)(context_params, apply_context, rng_key, jax_reps, context_set, fringe_sizes)
print (grad_log_context, fringe_idx, contexts_by_fringe)

[]
{'context_policy/~/linear': {'b': DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0

In [23]:
target_context = contexts_by_fringe[fringe_idx][0]
target_goal = target_context["polished"]["goal"]
target_representation = jax_reps[context_set.index(target_context)]
jnp.expand_dims(target_representation,0).shape

(1, 256)

In [24]:
MORE_TACTICS = True
if not MORE_TACTICS:
    thms_tactic = ["simp", "fs", "metis_tac"]
    thm_tactic = ["irule"]
    term_tactic = ["Induct_on"]
    no_arg_tactic = ["strip_tac"]
else:
    thms_tactic = ["simp", "fs", "metis_tac", "rw"]
    thm_tactic = ["irule", "drule"]
    term_tactic = ["Induct_on"]
    no_arg_tactic = ["strip_tac", "EQ_TAC"]
    
tactic_pool = thms_tactic + thm_tactic + term_tactic + no_arg_tactic
print (len(tactic_pool))

9


In [25]:
init_tac, apply_tac = hk.transform(policy_networks._tac_forward)

tactic_params = init_tac(rng_key, jnp.expand_dims(target_representation,0), len(tactic_pool))
apply_tac = partial(jax.jit, static_argnums=3)(apply_tac)


In [26]:
#takes a goal encoding and samples tactic from network, and returns log prob for gradient 
def sample_tactic(tactic_params, tac_net, rng_key, goal_endcoding, action_size=len(tactic_pool)):
    tac_probs = tac_net(tactic_params, rng_key, goal_endcoding, action_size)[0]
    tac_idx = random.categorical(rng_key, tac_probs)
    log_prob = jnp.log(tac_probs[tac_idx])
    return log_prob, tac_idx

grad_log_tac, tac_idx = jax.grad(sample_tactic, has_aux=True)(tactic_params, apply_tac, rng_key, jnp.expand_dims(target_representation,0), len(tactic_pool))

sampled_tac = tactic_pool[tac_idx]

In [27]:
#for testing

#if tactic requires no argument
if sampled_tac in no_arg_tactic:
    full_tactic = tactic_pool[tac]
    
#Induct_on case; use term policy to find which term to induct on 
elif sampled_tac in term_tactic:
    goal_tokens = target_goal.split()
    term_tokens = [[t] for t in set(goal_tokens) if t[0] == "V"]
    #add conditional if tokens is empty 
    
    

In [28]:
goal_tokens = target_goal.split()
#goal_tokens = dict.fromkeys(goal_tokens): set should be better?
term_tokens = [[t] for t in set(goal_tokens) if t[0] == "V"]

print (term_tokens)

[['Vh'], ['Vt']]


In [29]:
#now want encodings for terms from AE

term_reps = []

for term in term_tokens:
    term_rep, _ = batch_encoder_.encode([term])
    #output is bidirectional so concat vectors
    term_reps.append(torch.cat(term_rep.split(1), dim=2).squeeze(0))

# convert to jax
term_reps = jnp.stack([jnp.array(term_reps[i][0]) for i in range(len(term_reps))])
term_reps

DeviceArray([[-1.50797427e-01, -4.05441016e-01,  8.26748386e-02,
              -8.99273902e-02,  1.08627982e-01,  5.55764198e-01,
               3.58997732e-02, -1.37602121e-01,  1.11121483e-01,
               6.91576898e-01, -3.02194536e-01,  1.28379688e-01,
               4.34740540e-03, -1.77832589e-01, -1.81474805e-01,
              -5.11666089e-02,  2.18712777e-01,  1.82037473e-01,
              -1.76419318e-01, -1.74376853e-02, -2.81876445e-01,
              -7.17407316e-02, -9.03564021e-02,  4.05178964e-01,
               2.21685514e-01, -2.12503389e-01, -1.25464082e-01,
              -2.09619291e-02,  4.00536537e-01, -1.25789627e-01,
               1.43232912e-01, -3.69057447e-01,  1.10187300e-01,
               4.06071633e-01, -3.17936167e-02, -1.03816194e-02,
               1.77889511e-01,  3.11252624e-02,  6.45184703e-03,
              -5.59998453e-02, -1.75853278e-02, -2.28942968e-02,
              -1.39597759e-01, -2.39227176e-01,  1.30884767e-01,
              -2.72831082

In [30]:
# now want inputs to term_net to be target_representation (i.e. goal) concatenated with terms
# models the policies conditional dependence of the term given the goal

#stack goal representation for each token
goal_stack = jnp.concatenate([jnp.expand_dims(target_representation,0) for _ in term_tokens])

#concat with term encodings to give candidate matrix
candidates = jnp.concatenate([goal_stack, term_reps], 1)

print (candidates.shape)

(2, 512)


In [31]:
# init_term, apply_term = hk.transform(policy_networks._term_forward)

# #TODO why do we need the tactic? The use of term net implicitly assumes we are using induct_on, and every time this is called the tactic index will be the same?

# term_params = init_term(rng_key, candidates, tac_idx, len(tactic_pool), candidates.shape[1])
# apply_term = partial(jax.jit, static_argnums=(4,5))(apply_term)


In [32]:
# #takes a goal encoding and samples tactic from network, and returns log prob for gradient 
# def sample_term(term_params, term_net, rng_key, candidates, tactic,TAC_SIZE, MAX_LEN):
#     term_scores = term_net(term_params, rng_key, candidates, tactic, TAC_SIZE, MAX_LEN)
#     term_scores = jnp.ravel(term_scores)
#     term_probs = jax.nn.softmax(term_scores)
#     term_idx = random.categorical(rng_key, term_probs)
#     log_prob = jnp.log(term_probs[term_idx])
#     return log_prob, term_idx

# grad_log_term, tac_term = jax.grad(sample_term, has_aux=True)(term_params, apply_term, rng_key, candidates, tac_idx, len(tactic_pool), candidates.shape[1])

# print (grad_log_term, tac_term)

In [33]:
#term policy implementation without tactic as an argument (since it will always be Induct_on)
#policy will now learn a mapping for V(term | goal)

init_term, apply_term = hk.transform(policy_networks._term_no_tac_forward)
term_params = init_term(rng_key, candidates)
apply_term = jax.jit(apply_term)

print (candidates.shape)

(2, 512)


In [34]:
def sample_term(term_params, term_net, rng_key, candidates):
    term_scores = term_net(term_params, rng_key, candidates)
    term_scores = jnp.ravel(term_scores)
    term_probs = jax.nn.softmax(term_scores)
    term_idx = random.categorical(rng_key, term_probs)
    log_prob = jnp.log(term_probs[term_idx])
    return log_prob, term_idx

grad_log_term, term_idx = jax.grad(sample_term, has_aux=True)(term_params, apply_term, rng_key, candidates)#, tac_idx, len(tactic_pool), candidates.shape[1])

#print (grad_log_term, term_idx)

In [35]:
sampled_term = term_tokens[term_idx]

tm = sampled_term[0][1:] # remove headers, e.g., "V" / "C" / ...

if tm:
    tactic = "Induct_on `{}`".format(tm)
else:
    # only to raise an error
    tactic = "Induct_on"

In [36]:
#get candidate arguments

with open("polished_def_dict.json") as f:
    defs = json.load(f)

fact_pool = list(defs.keys())
fact_pool

['@ @ Cbool$/\\ @ Cbool$! | Vl2 @ @ Cmin$= @ Clist$ZIP @ @ Cpair$, Clist$NIL Vl2 Clist$NIL @ @ Cbool$/\\ @ Cbool$! | Vl1 @ @ Cmin$= @ Clist$ZIP @ @ Cpair$, Vl1 Clist$NIL Clist$NIL @ Cbool$! | Vx1 @ Cbool$! | Vl1 @ Cbool$! | Vx2 @ Cbool$! | Vl2 @ @ Cmin$= @ Clist$ZIP @ @ Cpair$, @ @ Clist$CONS Vx1 Vl1 @ @ Clist$CONS Vx2 Vl2 @ @ Clist$CONS @ @ Cpair$, Vx1 Vx2 @ Clist$ZIP @ @ Cpair$, Vl1 Vl2',
 '@ @ Cbool$/\\ @ @ Cmin$= @ Clist$UNZIP Clist$NIL @ @ Cpair$, Clist$NIL Clist$NIL @ Cbool$! | Vx @ Cbool$! | Vl @ @ Cmin$= @ Clist$UNZIP @ @ Clist$CONS Vx Vl @ @ Cpair$, @ @ Clist$CONS @ Cpair$FST Vx @ Cpair$FST @ Clist$UNZIP Vl @ @ Clist$CONS @ Cpair$SND Vx @ Cpair$SND @ Clist$UNZIP Vl',
 '@ Cbool$! | Ve @ Cbool$! | VL @ @ Cmin$= @ @ Clist$UNIQUE Ve VL @ Cbool$? | VL1 @ Cbool$? | VL2 @ @ Cbool$/\\ @ @ Cmin$= @ @ Clist$APPEND @ @ Clist$APPEND VL1 @ @ Clist$CONS Ve Clist$NIL VL2 VL @ @ Cbool$/\\ @ Cbool$~ @ @ Cbool$IN Ve @ Clist$LIST_TO_SET VL1 @ Cbool$~ @ @ Cbool$IN Ve @ Clist$LIST_TO_SET VL2',
 '@

In [37]:
encoded_database = torch.load('encoded_include_probability.pt')

encoded_fact_pool = torch.index_select(encoded_database, 0, torch.tensor(allowed_arguments_ids))
encoded_fact_pool = jnp.array(encoded_fact_pool)

In [38]:
# argument network for tactics with one or multiple arguments

   
hidden = jnp.expand_dims(target_representation,0)
# hidden = (hidden0, hidden1)

# concatenate the candidates with hidden states (i.e. goal).

goal_stack = jnp.concatenate([jnp.expand_dims(target_representation,0) for _ in encoded_fact_pool])


candidates = jnp.concatenate([encoded_fact_pool, goal_stack], 1)


init_state = hk.LSTMState(hidden, hidden)

#init_state

In [39]:
init_arg, apply_arg = hk.transform(policy_networks._arg_forward)
arg_params = init_arg(rng_key, random.randint(rng_key, (), 0, len(tactic_pool)), random.normal(rng_key,(1,512)), init_state, len(tactic_pool), 256)

apply_arg = partial(jax.jit, static_argnums=(5,6))(apply_arg)


In [40]:
#out_arg = apply_arg(arg_params,  rng_key, tac_idx, candidates, init_state, len(tactic_pool), 256)

#function for sampling single argument given previous arguments, 
def sample_arg(arg_params, arg_net, rng_key, input_, candidates, hidden, tactic_size, embedding_dim):
    hidden, arg_scores = arg_net(arg_params, rng_key, input_, candidates, hidden, tactic_size, embedding_dim)
    arg_scores = jnp.ravel(arg_scores)
    arg_probs = jax.nn.softmax(arg_scores)
    arg_idx = random.categorical(rng_key, arg_probs)
    log_prob = jnp.log(arg_probs[arg_idx])
    return log_prob, (arg_idx, hidden)

In [41]:
#print (init_state)
# run through first with tac_idx to initialise state with tactic as c_0
_, (_, hidden) = jax.grad(sample_arg, has_aux=True)(arg_params, apply_arg, rng_key, tac_idx, candidates, init_state, len(tactic_pool), 256)
#print (hidden)t
# Q: Do we need to make the hidden state the goal again, now with updated cell from first pass? or is new hidden fine?

In [42]:
#log gradient of context policy for all arguments will be sum of log gradients of each argument.
#hence can just take the sum over gradients one at a time 

#should we add an argument terminating character or just let it run to arg_len?


# if tactic_pool[true_tac] in thm_tactic:
#     arg_len = 1
# else:
#     arg_len = ARG_LEN

def run_arg(arg_params, apply_arg, rng_key, input_, candidates, hidden, tac_size, encoding_size):
    gradients = []
    arg_inds = []
    ARG_LEN = 5
    input_ = tac_idx
    for _ in range(ARG_LEN):
        grad_log_arg, (arg_idx, hidden) = jax.grad(sample_arg, has_aux=True)(arg_params, apply_arg, rng_key, input_, candidates, hidden, len(tactic_pool), 256)
        gradients.append(grad_log_arg)
        arg_inds.append(arg_idx)
        input_ = jnp.expand_dims(encoded_fact_pool[arg_idx], 0)
        #renewing candidates sems redundant from old code?
    return gradients, arg_inds

run_arg(arg_params, apply_arg, rng_key, tac_idx, candidates, hidden, len(tactic_pool), 256)


([{'arg_policy/~/embed': {'embeddings': DeviceArray([[0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 ...,
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)},
   'arg_policy/~/linear': {'b': DeviceArray([ 9.47159599e-04,  3.64837679e-03,  6.05252059e-03,
                 -1.29515640e-04, -3.42582236e-04, -6.27101457e-04,
                 -4.03735694e-03, -1.49760128e-03, -7.43823533e-04,
                 -1.66814644e-02, -5.75052691e-04, -9.40903556e-03,
                 -6.07361365e-03, -1.27711904e-03,  2.38784181e-04,
                  1.35620103e-05, -4.68308385e-03, -1.41876172e-02,
                 -4.28223808e-04,  1.35205092e-03, -8.63559172e-03,
                 -2.16048490e-02, -2.01686489e-04, -9.20210872e-03,
                 -4.67421440e-03, -1.19106036e-04,  0.00000000e+

In [43]:
#can multiply learning rate by reward for each update to give the policy gradient after grad of log probs is done 
context_lr = 1e-2
tactic_lr = 1e-2 
arg_lr = 1e-2
term_lr = 1e-2

context_optimiser = optax.rmsprop(context_lr)
tactic_optimiser = optax.rmsprop(tactic_lr)
arg_optimiser = optax.rmsprop(arg_lr)
term_optimiser = optax.rmsprop(term_lr)

opt_state_context = context_optimiser.init(context_params)
opt_state_tactic = tactic_optimiser.init(tactic_params)
opt_state_arg = arg_optimiser.init(arg_params)
opt_state_term = term_optimiser.init(term_params)




In [44]:
#test updating parameters by multiplying learning rate with return

test_return = 5

scaled_lr = term_lr * test_return 

term_optimiser = optax.rmsprop(scaled_lr)

opt_state_term = term_optimiser.init(term_params)


updates, opt_state_term = context_optimiser.update(grad_log_term, opt_state_term)
term_params = optax.apply_updates(term_params, updates)

print (opt_state_term)

new_return = 8

scaled_lr_new = term_lr * new_return 

new_term_optimiser = optax.rmsprop(scaled_lr_new)

#keep old state but apply to new optimiser lr

updates, opt_state_term = new_term_optimiser.update(grad_log_term, opt_state_term)
term_params = optax.apply_updates(term_params, updates)


(ScaleByRmsState(nu={'term_policy_no_tac/~/linear': {'b': DeviceArray([3.04095156e-05, 0.00000000e+00, 1.80568776e-07,
             2.51793852e-09, 3.96345825e-07, 0.00000000e+00,
             1.00831057e-05, 9.97164807e-06, 1.88952856e-06,
             0.00000000e+00, 0.00000000e+00, 3.29635043e-07,
             1.90811306e-06, 0.00000000e+00, 0.00000000e+00,
             1.67113433e-06, 1.53847179e-06, 0.00000000e+00,
             0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
             0.00000000e+00, 2.07858488e-06, 0.00000000e+00,
             0.00000000e+00, 2.56541375e-06, 0.00000000e+00,
             0.00000000e+00, 6.60825481e-06, 2.78377797e-06,
             2.30452042e-05, 1.48913859e-09, 3.42463827e-05,
             8.95741294e-08, 1.12436401e-05, 2.85953456e-06,
             1.62372851e-06, 0.00000000e+00, 1.35026127e-07,
             6.44317261e-05, 5.83088149e-06, 4.20786409e-06,
             1.90640640e-05, 6.09186509e-06, 1.80534538e-08,
             7.50922845e-06

In [45]:
#can multiply learning rate by reward for each update to give the policy gradient after grad of log probs is done 
# context_lr = 1e-2
# # tactic_lr = 1e-2 
# # arg_lr = 1e-2
# # term_lr = 1e-2

# context_optimiser = optax.rmsprop(context_lr)
# # tactic_optimiser = optax.rmsprop(tactic_lr)
# # arg_optimiser = optax.rmsprop(arg_lr)
# # term_optimiser = optax.rmsprop(term_lr)

# opt_state_context = context_optimiser.init(context_params)
# # opt_state_tactic = tactic_optimiser.init(tactic_params)
# # opt_state_arg = arg_optimiser.init(arg_params)
# # opt_state_term = term_optimiser.init(term_params)





#gradient example. May need to construct separate function for each net during training 

# def compute_probs(params, net, *args):
#     probs = jax.nn.softmax(jnp.ravel(net(params,*args)))
#     logits = jnp.log(probs)
#     ind = random.categorical(rng_key, logits)
#     log_prob = logits[ind]
#     return log_prob

# grad = jax.grad(compute_probs)(context_params, apply_context, rng_key, jax_reps)#c_term, x_arg,TAC_SIZE, MAX_LEN)

# # updates, opt_state_term = context_optimiser.update(grad, opt_state_context)
# context_params = optax.apply_updates(context_params, updates)


# context_params
